Change directory and mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# cd into the independent project folder
cd /content/drive/My Drive/FourthBrain/independent project

In [ ]:
# TRIED TO DOWNLOAD THE DATA SET DIRECTLY FROM KAGGLE BUT IT KEPT ON TIMING OUT

# ! pip install -q kaggle
#from google.colab import files
#files.upload()
#! mkdir ~/.kaggle (using the one in .kaggle instead)
#! cp kaggle.json  ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets list

#! mv *.jpg data/
#!cd data
#!kaggle competitions download -c state-farm-distracted-driver-detection 
# download the data (about 4GB... good thing i have 100GB), but keeps on timing out so downloading from website
# updating kaggle (also how do i run in an venv instead of from root?)
#!pip uninstall -y kaggle
#!pip install --upgrade pip
#!pip install kaggle==1.5.6


# downloaded zip
#!kaggle competitions download -c state-farm-distracted-driver-detection  
# unzip file: 
# !unzip "/content/drive/My Drive/FourthBrain/independent project/data/state-farm-distracted-driver-detection.zip" -d "/content/drive/My Drive/FourthBrain/independent project/data/statefarm"
# manually removed zip file from drive
# NOT SHOWING UP!!

Trying to familiarize myself with the data:

1. Looking at csv file with all the labels for training data
2. Adding filepath of images to dataframe


In [4]:
import pandas as pd
import numpy as np

In [23]:
csv_filename = 'data/state-farm-distracted-driver-detection/driver_imgs_list.csv'
df = pd.read_csv(csv_filename)
print(df)

      subject classname            img
0        p002        c0  img_44733.jpg
1        p002        c0  img_72999.jpg
2        p002        c0  img_25094.jpg
3        p002        c0  img_69092.jpg
4        p002        c0  img_92629.jpg
...       ...       ...            ...
22419    p081        c9  img_56936.jpg
22420    p081        c9  img_46218.jpg
22421    p081        c9  img_25946.jpg
22422    p081        c9  img_67850.jpg
22423    p081        c9   img_9684.jpg

[22424 rows x 3 columns]


Get image paths and also change the class name to an integer

In [31]:
# images of safe driving (going to try to load the first image)
#classname = 'c0'
#filenames = df[df['classname'] == classname]['img']

# training images are in a special folder structure
def get_img_path(classname, filename):
  return 'data/state-farm-distracted-driver-detection/imgs/train/' + classname + '/' + filename

# make sure it works:
# print(get_img_path(classname, filenames[1]))
df['img_path'] = get_img_path(df['classname'], df['img'])
df['class']=df['classname'].apply(lambda x: int(x[1]))
df.head()

,subject,classname,img,img_path,class
0,p002,c0,img_44733.jpg,data/state-farm-distracted-driver-detection/im...,0
1,p002,c0,img_72999.jpg,data/state-farm-distracted-driver-detection/im...,0
2,p002,c0,img_25094.jpg,data/state-farm-distracted-driver-detection/im...,0
3,p002,c0,img_69092.jpg,data/state-farm-distracted-driver-detection/im...,0
4,p002,c0,img_92629.jpg,data/state-farm-distracted-driver-detection/im...,0


Experiment with reading in the image data (many ways to do this)

https://www.pluralsight.com/guides/importing-image-data-into-numpy-arrays

In [32]:
df.iloc[0,3] # first image in the dataframe

'data/state-farm-distracted-driver-detection/imgs/train/c0/img_44733.jpg'

In [37]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
from matplotlib import image
from PIL import Image

In [40]:
mydir = !pwd
im_test = load_img(mydir[0]+'/'+df.iloc[0,3]) # unzipped files are not showing up in google drive... maybe wait a bit

In [42]:
# try getting the test images instead 
#img_file = 'data/state-farm-distracted-driver-detection/imgs/test/img_13953.jpg'
#imag
#imtest = load_img(img_file) # this is taking forever!
imarray = img_to_array(im_test)
imarray.shape

(480, 640, 3)

In [ ]:
# also try reading using PIL lib

In [ ]:
gryscale = Image.open(img_file).convert('L') # grayscale # Taking forever because there are thousands of files in the drive! 
# gryscale.resize((320,240)) # scale down by 1/2
resize_dims = tuple([int(x/1) for x in (gryscale.size)])
gryscale.resize(resize_dims) # note: not resizing in place, gryscale still original size

OSError: ignored

In [ ]:
# convert to numpy array
np.array(gryscale)
# convert img to row vector
np.array(gryscale).reshape(1,np.array(gryscale).size) 

array([[43, 43, 44, ..., 21, 20, 17]], dtype=uint8)

Function to take in a filepath and output a rescaled image in grayscale as a vector

In [ ]:
def img2vec(filepath, resize_scale = 5, gray_scale = True):
  '''
  Loads image, converts to grayscale, downsamples by resize_scale and returns a row vector 
  To keep color image, set gray_scale = True 
  To keep the original size of the image, set resize_scale= 1
  '''
  im = Image.open(filepath)
  if gray_scale:
    im = im.convert('L')
  if resize_scale > 1:
    resize_dims = tuple([int(x/resize_scale) for x in (gryscale.size)])
    im = im.resize(resize_dims)
  return np.array(im).reshape(1, np.array(im).size)
    